In [3]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=2efb00431ca4fe1427f0cb54b33c12a96f9d144c758053cf5f7416c7c241cd58
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .getOrCreate()
    
sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=pyspark-shell>

# PySpark cheatsheet

La API de Spark tiene varias funciones, pero en este _notebook_ vamos a presentar las funciones más importantes de RDDs. Un RDD es un Resilient Distributed Dataset, que es la estructura básica de Apache Spark para trabajar con datos distribuidos. Estos datos se cargan (en general) desde un sistema de archivos distribuidos, por lo que podemos suponer que en un nodo de cómputo no se encuentran todos los datos, por lo que tenemos que usar estas funciones pensadas para trabajar en entornos distribuidos. Puedes ver la API completa [aquí](https://spark.apache.org/docs/latest/api/python/reference/pyspark.html#rdd-apis).


### 1. Map

La función `map` retorna un nuevo RDD al que se le aplica una función en cada posición.

In [5]:
rdd = sc.parallelize(["b", "a", "c"])
rdd.map(lambda x: (x, 1)).collect()

[('b', 1), ('a', 1), ('c', 1)]

### 2. Reduce

La función `reduce` pasa de un RDD a un único elemento

In [6]:
rdd = sc.parallelize([1, 2, 3, 4, 5])
rdd.reduce(lambda x, y: x + y)

15

### 3. ReduceByKey

La función `reduceByKey` se encarga de formar un iterable para cada llave de un RDD. Luego se aplica la función reduce para cada iterable.

In [7]:
rdd = sc.parallelize([("a", 1), ("b", 1), ("a", 1)])
rdd.reduceByKey(lambda x, y: x + y).collect()

[('b', 1), ('a', 2)]

### 4. Count

La función `count` cuenta los elementos en un RDD.

In [8]:
sc.parallelize([2, 3, 4]).count()

3

In [9]:
sc.parallelize([2, 3, 4, 4]).count()

4

### 5. Distinct

La función `distinct` retorna los elementos distintos de una colección.

In [11]:
sc.parallelize([1, 1, 2, 3]).distinct().collect()

[2, 1, 3]

In [12]:
sc.parallelize([(1, 1), (1, 2), (1, 1)]).distinct().collect()

[(1, 1), (1, 2)]

### 6. Filter

La función `filter`  aplica una función _booleana_, y deja solo los elementos que retornen `True` en la función.

In [13]:
rdd = sc.parallelize([1, 2, 3, 4, 5])
rdd.filter(lambda x: x % 2 == 0).collect()

[2, 4]

### 7. Join

La función `join` nos permite hacer un _join_ según las llaves de dos RDD.

In [14]:
rdd1 = sc.parallelize([("a", 1), ("b", 4)])
rdd2 = sc.parallelize([("a", 2), ("a", 3)])

rdd1.join(rdd2).collect()

[('a', (1, 2)), ('a', (1, 3))]

### 8. LeftJoin

La función `leftOuterJoin` hace un _left outer join_ entre dos RDDs. Al igual que en una BD relacional, el orden importa.

In [16]:
rdd1 = sc.parallelize([("a", 1), ("b", 4)])
rdd2 = sc.parallelize([("a", 2)])

rdd1.leftOuterJoin(rdd2).collect()

[('b', (4, None)), ('a', (1, 2))]

### 9. Keys

La función `keys` nos retorna las llaves en un RDD que representa pares key-value.

In [17]:
rdd = sc.parallelize([(1, 2), (3, 4)]).keys()
rdd.collect()

[1, 3]

### 10. Values

La función `values` nos retorna los valores en un RDD que representa pares key-value.

In [18]:
rdd = sc.parallelize([(1, 2), (3, 4)]).values()
rdd.collect()

[2, 4]

### 11. FlatMap

La función `flatMap` permite ejecutar la función `map` extrayendo los elementos de los iterables que genera.

In [19]:
rdd = sc.parallelize([2, 3, 4])
rdd.flatMap(lambda x: range(1, x)).collect()

[1, 1, 2, 1, 2, 3]

Ahora veamos qué retorna la función `map` común.

In [20]:
rdd = sc.parallelize([2, 3, 4])
rdd.map(lambda x: range(1, x)).collect()

[range(1, 2), range(1, 3), range(1, 4)]

Otro ejemplo de `flatMap` es este.

In [21]:
rdd = sc.parallelize([2, 3, 4])
rdd.flatMap(lambda x: [(x, x), (x, x)]).collect()

[(2, 2), (2, 2), (3, 3), (3, 3), (4, 4), (4, 4)]

### 12. Zip

La función `zip` permite juntar dos `RDD` en uno solo.

In [22]:
rdd1 = sc.parallelize([1, 2, 3, 4, 5])
rdd2 = sc.parallelize([6, 7, 8, 9, 10])

rdd1.zip(rdd2).collect()

[(1, 6), (2, 7), (3, 8), (4, 9), (5, 10)]

## 13. AggregateByKey

La función AggregateByKey es similar a la función `reduceByKey`, pero permite hacer operaciones más complejas. La idea es agregar en cada partición local, partiendo de un valor inicial comunmente conocido como el neutro. Luego se agregan los resultados locales con una función que combina los resultados por partición.

In [28]:
rdd = sc.parallelize([("a", 1), ("b", 1), ("a", 2)])

# Para cada key, vamos ejecutando la función de agregación entre pares
# partiendo desde el elemento inicial

# Función conmutativa, no podemos asumir orden
seqFunc = (lambda x, y: (x[0] + y, x[1] + 1))
combFunc = (lambda x, y: (x[0] + y[0], x[1] + y[1]))
rdd.aggregateByKey(
  (0, 0), # Elemento inicial
  seqFunc, # Función de agregación local
  combFunc # Función para combinar resultados locales
).collect()

[('a', (3, 2)), ('b', (1, 1))]

Lo que estamos haciendo para cada llave es calcular la suma de todos sus elementos, junto al número de elementos. En este ejemplo valor inicial es `(0, 0)`. Así que analicemos el caso para la llave `a`. Supongamos que tenemos el iterable en el orden `[2, 1]`, la función de agregación hace lo siguiente:

1. Primero generamos la tupla `(x[0] + y, x[1] + 1)`, donde `x` es la tupla con el valor inicial e `y` el primer valor del iterable. Por lo tanto hacemos `(0 + 2, 0 + 1)`.
2. Luego `x` tiene el valor del acumulado hasta ahora. Así que la siguiente operación es: `(2 + 1, 1 + 1)`.

Ahora, si no todos los valores de `a` están en el mismo nodo de cómputo, estos se comunicarán después de la agregación local. La idea de esta forma de computar cosas es que minimizamos los cómputos entre servidores.

### 14. MapValues

Ejecutamos un `map` para todos los _values_ de un RDD que almacena pares key-value.

In [33]:
rdd = sc.parallelize([("a", ["apple", "banana", "lemon"]), ("b", ["grapes"])])
rdd.mapValues(lambda x: len(x)).collect()

[('a', 3), ('b', 1)]

**Ojo**. Si el valor es un iterable, no aplicamos la función a cada elemento del iterable, sino al iterable entero. Esto es porque estamos mapeando todos los _values_ del RDD. Tampoco se agrupa por cada llave.

In [34]:
rdd = sc.parallelize([("a", ["apple", "banana", "lemon"]), ("b", ["grapes"]), ("a", ["kiwi"])])
rdd.mapValues(lambda x: len(x)).collect()

[('a', 3), ('b', 1), ('a', 1)]

### 15. GroupByKey

La función `groupByKey` permite juntar todos los elementos para cada llave. **Ojo**. Se debe privilegiar el uso de `reduceByKey` o `aggregateByKey` porque tienen mucho mejor desempeño.

In [31]:
rdd = sc.parallelize([("a", 1), ("b", 1), ("a", 2)])
rdd.groupByKey().collect()

[('b', <pyspark.resultiterable.ResultIterable at 0x7fd492b55510>),
 ('a', <pyspark.resultiterable.ResultIterable at 0x7fd492b55a80>)]

Para poder hacer algo con los elementos podemos hacer lo siguiente.

In [32]:
rdd = sc.parallelize([("a", 1), ("b", 1), ("a", 2)])
rdd.groupByKey().mapValues(list).collect()

[('b', [1]), ('a', [1, 2])]

Veamos otro ejemplo.

In [35]:
rdd = sc.parallelize([("a", 1), ("b", 1), ("a", 2)])
rdd.groupByKey().mapValues(len).collect()

[('b', 1), ('a', 2)]

### Bonus: la función Broadcast

La función `broadcast` no es de la API de RDD, sino que se asocia al _Spark Context_. Permite crear una variable accesible para todos los nodos de cómputo.

In [26]:
mapping = {1: 10001, 2: 10002}
bc = sc.broadcast(mapping)

rdd = sc.parallelize([1, 2, 3, 4, 5])
rdd2 = rdd.map(lambda x: bc.value[x] if x in bc.value else -1)

rdd2.collect()

[10001, 10002, -1, -1, -1]

Recordemos que la idea es hacer _broadcasting_ de variables pequeñas. También podemos destruirlas.

In [27]:
bc.destroy()